Copyright 2016 Google Inc. All Rights Reserved.

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

    http://www.apache.org/licenses/LICENSE-2.0
Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.

--------------------------------------

This notebook is similar in functionality to [this python script](https://github.com/amygdala/tensorflow-workshop/blob/master/workshop_sections/mnist_series/mnist_tflearn.py), and is used with [this README](https://github.com/amygdala/tensorflow-workshop/blob/master/workshop_sections/mnist_series/02_README_mnist_tflearn.md).  It shows how to use TensorFlow's high-level Estimator classes to easily build a classifier with multiple hidden layers.

First, do some imports and set some variables:

In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import argparse
import os
import time

import numpy
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

# comment out for less info during the training runs.
tf.logging.set_verbosity(tf.logging.INFO)

In [2]:
# Set locations of data files
MNIST_DATA_DIR = "/tmp/MNIST_data"
FASHION_DATA_DIR = "/tmp/fashion-mnist"
# Select your choice of dataset
DATA_DIR = MNIST_DATA_DIR

# read in data, downloading first as necessary
DATA_SETS = input_data.read_data_sets(DATA_DIR)

# define a utility function for generating a new directory in which to save 
# model information, so multiple training runs don't stomp on each other.
def get_new_path(name=""):
    base="/tmp/tfmodels/mnist_estimators"
    logpath = os.path.join(base, name + "_" + str(int(time.time())))
    print("Logging to {}".format(logpath))
    return logpath

Extracting /tmp/MNIST_data/train-images-idx3-ubyte.gz
Extracting /tmp/MNIST_data/train-labels-idx1-ubyte.gz
Extracting /tmp/MNIST_data/t10k-images-idx3-ubyte.gz
Extracting /tmp/MNIST_data/t10k-labels-idx1-ubyte.gz


Next create an input function, using the `tf.train.shuffle_batch` function to take care of the batching and shuffling of the input data.

In [3]:
BATCH_SIZE = 40
# call with generate_input_fn(DATA_SETS.train) or generate_input_fn(DATA_SETS.test)

# These default settings will generate samples in the order of the file, forever.
def generate_input_fn(dataset, batch_size=BATCH_SIZE, shuffle=False, epochs=None):
    X = dataset.images
    Y = dataset.labels.astype(numpy.int64)
    return tf.estimator.inputs.numpy_input_fn(
        x={'pixels': X},
        y=Y,
        batch_size=batch_size,
        num_epochs=epochs,
        shuffle=shuffle
    )


We'll first define a function that adds a LinearClassifier and runs its `train()` method, which will train the model. Note that we didn't need to explicitly define a model graph or a training loop ourselves.  

Once we've trained the model, we run the `evaluate()` method, which uses the trained model. To do this, it loads the most recent checkpointed model info available.  The model checkpoint(s) will be generated during the training process.

In [4]:
def define_and_run_linear_classifier(num_steps, logdir, batch_size=BATCH_SIZE):
    """Run a linear classifier."""

    feature_columns = [tf.feature_column.numeric_column(
        "pixels", shape=784)]
    
    classifier = tf.estimator.LinearClassifier(
                feature_columns=feature_columns, 
                n_classes=10,
                model_dir=logdir
                )
    classifier.train(input_fn=generate_input_fn(DATA_SETS.train, 
                                                batch_size=batch_size,
                                                shuffle=True), 
                     steps=num_steps)
    
    print("Finished training.")
    
    # Evaluate accuracy.
    accuracy_score = classifier.evaluate(input_fn=generate_input_fn(
        DATA_SETS.test, batch_size, shuffle=False, epochs=1))['accuracy']
    
    print('Linear Classifier Accuracy: {0:f}'.format(accuracy_score))

Next, add a function that defines a `DNNClassifier`, and runs its `train()` method, which will train the model. Again note that we didn't need to explicitly define a model graph or a training loop ourselves.  

Then after we've trained the model, we run the classifier's `evaluate()` method, which uses the trained model. 

In [5]:
def define_and_run_dnn_classifier(num_steps, logdir, lr=.1, batch_size=40):
    """Run a DNN classifier."""
    feature_columns = [tf.feature_column.numeric_column(
        "pixels", shape=784)]
    
    classifier = tf.estimator.DNNClassifier(
        feature_columns=feature_columns, 
        n_classes=10,
        hidden_units=[200, 100, 50],
        optimizer=tf.train.ProximalAdagradOptimizer(learning_rate=lr),
        model_dir=logdir
        )
    # After you've done a training run with optimizer learning rate 0.1,
        # change it to 0.5 and run the training again.  Use TensorBoard to take
        # a look at the difference.  You can see both runs by pointing it to the
        # parent model directory, which by default is:
        #
        #   tensorboard --logdir=/tmp/tfmodels/mnist_estimators
        
    classifier.train(input_fn=generate_input_fn(DATA_SETS.train, 
                                                batch_size=batch_size,
                                                shuffle=True), 
                     steps=num_steps)

    print("Finished running the deep training via the train() method")
    
    accuracy_score = classifier.evaluate(input_fn=generate_input_fn(
        DATA_SETS.test, batch_size=batch_size, shuffle=False, epochs=1))['accuracy']

    print('DNN Classifier Accuracy: {0:f}'.format(accuracy_score))

Now we can call the functions that define and train our classifiers. (It takes a moment to set up the input data queue before the training starts).

Let's start with the LinearClassifier, which won't be very accurate. 


In [6]:
print("Running Linear classifier ...")
define_and_run_linear_classifier(num_steps=1000, 
                                 logdir=get_new_path("linear"), 
                                 batch_size=40)
# With 1000 steps and a batch size of 40, we see accuracy of approx 91% for MNIST

Running Linear classifier ...
Logging to /tmp/tfmodels/mnist_estimators/linear_1505717722
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_tf_random_seed': 1, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_save_checkpoints_steps': None, '_model_dir': '/tmp/tfmodels/mnist_estimators/linear_1505717722', '_save_summary_steps': 100}
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into /tmp/tfmodels/mnist_estimators/linear_1505717722/model.ckpt.
INFO:tensorflow:loss = 92.1034, step = 1
INFO:tensorflow:global_step/sec: 868.742
INFO:tensorflow:loss = 18.2605, step = 101 (0.116 sec)
INFO:tensorflow:global_step/sec: 698.48
INFO:tensorflow:loss = 13.0562, step = 201 (0.144 sec)
INFO:tensorflow:global_step/sec: 612.347
INFO:tensorflow:loss = 8.91486, step = 301 (0.163 sec)
INFO:tensorflow:global_step/sec: 621.018
INFO:tensorflow:lo

Now, let's run the DNN Classifier.  First, let's try it with a .1 learning rate.

In [7]:
print("Running DNN classifier with .1 learning rate...")
classifier = define_and_run_dnn_classifier(2000, 
                                           get_new_path("deep01"), 
                                           lr=.1)
# With 2000 steps and a batch size of 40, we see accuracy of approx 95% on MNIST

Running DNN classifier with .1 learning rate...
Logging to /tmp/tfmodels/mnist_estimators/deep01_1505717746
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_tf_random_seed': 1, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_save_checkpoints_steps': None, '_model_dir': '/tmp/tfmodels/mnist_estimators/deep01_1505717746', '_save_summary_steps': 100}
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into /tmp/tfmodels/mnist_estimators/deep01_1505717746/model.ckpt.
INFO:tensorflow:loss = 92.8653, step = 1
INFO:tensorflow:global_step/sec: 474.408
INFO:tensorflow:loss = 22.702, step = 101 (0.212 sec)
INFO:tensorflow:global_step/sec: 354.874
INFO:tensorflow:loss = 15.33, step = 201 (0.286 sec)
INFO:tensorflow:global_step/sec: 319.932
INFO:tensorflow:loss = 24.6402, step = 301 (0.311 sec)
INFO:tensorflow:global_step/sec: 322.672
IN

Let's loop through the training-eval loop a couple of times, so we get more accuracy readings. Make a for-loop and provide a stable path for your model, which will allow continuous training-eval loops.

In [ ]:
model_path = "/tmp/tfmodels/mnist_estimators/deep01_1505716594" # This is an example
for i in range(0,5):
    define_and_run_dnn_classifier(1000, model_path, lr=.1)

Let's see what MNIST and fashion-mnist look like side by side. Change the path of the DATA_DIR to point to your fashion-mnist dataset, and run the training again. Be sure to change your model path.

Now, let's run it with a .5 learning rate.

In [ ]:
print("Running DNN classifier with .5 learning rate...")
classifier = define_and_run_dnn_classifier(2000, 
                                           get_new_path("deep05"), 
                                           lr=.5)
# With 2000 steps and a batch size of 40, we see accuracy of approx 91%, though sometimes it does not converge at all.

To compare your results, start up TensorBoard as follows in a new terminal window. (If you get a 'not found' error, make sure you've activated your virtual environment in that new window):

```sh
$ tensorboard --logdir=/tmp/tfmodels/mnist_estimators
```
Or run the following (select Kernel --> Interrupt from the menu when you're done):

In [ ]:
!tensorboard --logdir=/tmp/tfmodels/mnist_estimators